# MegaDescriptor-S-224
- Configuration used for training MegaDescriptor-S-224 (https://huggingface.co/BVRA/MegaDescriptor-S-224)

In [5]:
from itertools import chain
import torch
import timm
import pandas as pd
import torchvision.transforms as T
from torch.optim import SGD
from wildlife_tools.data.dataset import WildlifeDataset
from wildlife_tools.data.split import SplitMetadata
from wildlife_tools.train.objective import ArcFaceLoss
from wildlife_tools.train.trainer import BasicTrainer


# Dataset configuration
metadata = pd.read_csv('../metadata/datasets/fca_base/metadata.csv') #changed to indiv dataset
image_root = '/Users/fmb/GitHub/764WildlifeReID/megadescriptor/data/images/size-256/fca_base'
transform = T.Compose([
    T.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
    T.RandAugment(num_ops=2, magnitude=20),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

dataset = WildlifeDataset(
    metadata = metadata, 
    root = image_root,
    split = SplitMetadata('split', 'train'),
    transform=transform
)

In [3]:
# !pip install pytorch-metric-learning

  Using cached pytorch_metric_learning-2.6.0-py3-none-any.whl.metadata (17 kB)


In [6]:
# Backbone and loss configuration
backbone = timm.create_model('swin_small_patch4_window7_224', num_classes=0, pretrained=True)
with torch.no_grad():
    dummy_input = torch.randn(1, 3, 224, 224)
    embedding_size = backbone(dummy_input).shape[1]
objective = ArcFaceLoss(num_classes=dataset.num_classes, embedding_size=embedding_size, margin=0.5, scale=64)

In [7]:
# Optimizer and scheduler configuration
params = chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)
min_lr = optimizer.defaults.get("lr") * 1e-3
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=min_lr)

In [9]:
# check if cuda is available
import torch

print("Is CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("Device count:", torch.cuda.device_count())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))

Is CUDA available: False
CUDA version: None
Device count: 0


AssertionError: Torch not compiled with CUDA enabled

In [8]:
# Setup training
trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    scheduler=scheduler,
    batch_size=64,
    accumulation_steps=2,
    num_workers=2,
    epochs=100,
    device='cuda',
)

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
trainer.train()